# Singular Value Decomposition 

### Loading the dataset 

In [1]:
# importing basic dependencies 
import numpy as np
import pandas as pd
import h5py

In [63]:
movies = pd.read_csv('../Data/movies_sml.csv')

In [64]:
movies.head()

,movieId,Title,genres,Actors,Director,Plot,Poster
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",John Lasseter,A cowboy doll is profoundly threatened and jea...,https://m.media-amazon.com/images/M/MV5BMDU2ZW...
1,2,Jumanji,Adventure|Children|Fantasy,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston,When two kids find and play a magical board ga...,https://m.media-amazon.com/images/M/MV5BZTk2Zm...
2,3,Grumpier Old Men,Comedy|Romance,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",Howard Deutch,John and Max resolve to save their beloved bai...,https://m.media-amazon.com/images/M/MV5BMjQxM2...
3,4,Waiting to Exhale,Comedy|Drama|Romance,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker,"Based on Terry McMillan's novel, this film fol...",https://m.media-amazon.com/images/M/MV5BYzcyMD...
4,5,Father of the Bride Part II,Comedy,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer,George Banks must deal not only with the pregn...,https://m.media-amazon.com/images/M/MV5BOTEyNz...


In [65]:
movies.shape

(11102, 7)

#### Note - initial data prep is in the ratings notebook in the ETL directory 

In [2]:
ratings_sml = pd.read_hdf('../Data/ratings_sml.h5')
ratings_sml.head()

,userId,num_user_rated,movieId,Title,rating
0,33455,150,110,Braveheart,5.0
1,33455,150,2959,Fight Club,4.5
2,33455,150,4226,Memento,4.5
3,33455,150,4878,Donnie Darko,4.5
4,33455,150,5577,Igby Goes Down,4.0


In [3]:
# creating matrix
ratings_matrix = ratings_sml.pivot(index='userId', columns='movieId', values='rating').fillna(0)
ratings_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,175991,175993,175995,175999,176001,176003,176007,176157,176211,176271
userId,,,,,,,,,,,,,,,,,,,,,
12,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,3.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
#compress ratings matrix
ratings_matrix.to_hdf('../Data/ratings_mx.h5', key='ratings_matrix',complib='blosc',complevel=9, mode='w' )


In [5]:
# normalize the data by each users mean and convert it from a dataframe to a numpy array.
R = ratings_matrix.values
user_ratings_mean = np.mean(R, axis = 1)
Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)


#### Setting up SVD 

In [10]:
# importing scipy denpendency 
from scipy.sparse.linalg import svds

# Compute the largest k singular values/vectors for a sparse matrix
U, sigma, Vt = svds(Ratings_demeaned, k = 50)


In [11]:
# convert sigma to a diagonal matrix 
sigma = np.diag(sigma)


#### Making Predictions from the Decomposed Matrices

In [12]:
#  add the user means back to get the actual star ratings prediction
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)


In [15]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = ratings_matrix.columns)
preds.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,175991,175993,175995,175999,176001,176003,176007,176157,176211,176271
0,2.560615,0.031274,-0.651091,0.020620,-0.533642,0.264343,-0.504495,-0.057654,-0.134222,0.201482,...,0.001771,0.001789,0.001807,0.001807,0.001807,0.001824,0.001754,0.001644,0.002967,0.002237
1,0.441066,-0.297052,0.211197,-0.065219,0.153935,2.553422,0.270768,-0.053733,0.102689,0.535472,...,-0.001425,-0.001077,-0.000729,-0.000729,-0.000729,-0.000382,-0.001773,-0.000501,-0.003033,-0.002046
2,-0.352039,-0.094756,-0.192779,0.064510,-0.023931,0.064820,-0.130847,0.012888,-0.102441,-0.709030,...,0.007978,0.007942,0.007906,0.007906,0.007906,0.007870,0.008014,0.009173,0.005091,0.006427
3,1.296987,1.336419,-0.154512,-0.097812,-0.045746,3.126836,-0.138816,-0.105895,0.094546,1.727142,...,-0.000140,-0.000042,0.000056,0.000056,0.000056,0.000154,-0.000238,0.002996,0.000401,-0.000531
4,4.743518,0.725185,-0.024024,-0.112855,0.097943,0.340626,0.199099,-0.030649,-0.011594,1.098070,...,0.001233,0.001167,0.001102,0.001102,0.001102,0.001036,0.001298,-0.000037,0.001855,0.002679


In [18]:
# converting dtype from float64 to float32 to make matrix smaller 
preds.astype(np.float32)
preds.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,175991,175993,175995,175999,176001,176003,176007,176157,176211,176271
0,2.560615,0.031274,-0.651091,0.020620,-0.533642,0.264343,-0.504495,-0.057654,-0.134222,0.201482,...,0.001771,0.001789,0.001807,0.001807,0.001807,0.001824,0.001754,0.001644,0.002967,0.002237
1,0.441066,-0.297052,0.211197,-0.065219,0.153935,2.553422,0.270768,-0.053733,0.102689,0.535472,...,-0.001425,-0.001077,-0.000729,-0.000729,-0.000729,-0.000382,-0.001773,-0.000501,-0.003033,-0.002046
2,-0.352039,-0.094756,-0.192779,0.064510,-0.023931,0.064820,-0.130847,0.012888,-0.102441,-0.709030,...,0.007978,0.007942,0.007906,0.007906,0.007906,0.007870,0.008014,0.009173,0.005091,0.006427
3,1.296987,1.336419,-0.154512,-0.097812,-0.045746,3.126836,-0.138816,-0.105895,0.094546,1.727142,...,-0.000140,-0.000042,0.000056,0.000056,0.000056,0.000154,-0.000238,0.002996,0.000401,-0.000531
4,4.743518,0.725185,-0.024024,-0.112855,0.097943,0.340626,0.199099,-0.030649,-0.011594,1.098070,...,0.001233,0.001167,0.001102,0.001102,0.001102,0.001036,0.001298,-0.000037,0.001855,0.002679


In [46]:
preds.shape


(27585, 11102)

In [47]:
# converting negative values to zero 
preds[preds < 0] = 0

In [48]:
preds.to_hdf('../Data/predsfin_hdf.h5', key='preds',complib='blosc',complevel=9, mode='w')

In [49]:
preds.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,175991,175993,175995,175999,176001,176003,176007,176157,176211,176271
0,2.560615,0.031274,0.000000,0.02062,0.000000,0.264343,0.000000,0.000000,0.000000,0.201482,...,0.001771,0.001789,0.001807,0.001807,0.001807,0.001824,0.001754,0.001644,0.002967,0.002237
1,0.441066,0.000000,0.211197,0.00000,0.153935,2.553422,0.270768,0.000000,0.102689,0.535472,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.06451,0.000000,0.064820,0.000000,0.012888,0.000000,0.000000,...,0.007978,0.007942,0.007906,0.007906,0.007906,0.007870,0.008014,0.009173,0.005091,0.006427
3,1.296987,1.336419,0.000000,0.00000,0.000000,3.126836,0.000000,0.000000,0.094546,1.727142,...,0.000000,0.000000,0.000056,0.000056,0.000056,0.000154,0.000000,0.002996,0.000401,0.000000
4,4.743518,0.725185,0.000000,0.00000,0.097943,0.340626,0.199099,0.000000,0.000000,1.098070,...,0.001233,0.001167,0.001102,0.001102,0.001102,0.001036,0.001298,0.000000,0.001855,0.002679


In [50]:
preds.shape


(27585, 11102)

In [73]:
# build a fuction to retun  the higest predicted rating + merge to 
def recommend_movies(predictions, userId, movies, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = userId - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    #sorted_user_predictions = preds[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.userId == (userId)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )
    rec_movies = recommendations['Title']
    already_rated = user_full['Title_x']
    return  already_rated, rec_movies

In [74]:
# testing function - recommend 20 movies for user id 10000
already_rated, predictions = recommend_movies(preds, 10000, movies, ratings_sml, 20)


In [75]:
predictions

1106                              Good Will Hunting
1658                                American Beauty
467                      Terminator 2: Judgment Day
800                                      Goodfellas
254                          Léon: The Professional
2229                                        Memento
752                                  Reservoir Dogs
255                                    Pulp Fiction
102                                      Braveheart
1698                                     Fight Club
1160                             As Good as It Gets
1424                             American History X
1048                              L.A. Confidential
1265                   There's Something About Mary
977     Austin Powers: International Man of Mystery
488                                           Fargo
877                                   Jerry Maguire
1946                                      Gladiator
2709                                Minority Report
2132        

In [69]:
already_rated

Series([], Name: Title_x, dtype: object)